In [1]:
import json
from pyspark import SparkContext

In [2]:
path = "/Users/liangsiqi/Documents/Dataset/yelp_dataset/"
user_file = "user.json"
review_file = "review.json"
business_file = "business.json"

In [3]:
task1_json_answer = {'total_users':  1637138,
                     'avg_reviews':  22.287697799452484,
                     'distinct_usernames':  124917,
                     'num_users':  168467,
                     'top10_popular_names':[['John', 14430],
                                            ['David', 13616],
                                            ['Michael', 13559],
                                            ['Chris', 12178],
                                            ['Mike', 11358],
                                            ['Jennifer', 10360],
                                            ['Jessica', 9349],
                                            ['Sarah', 8748],
                                            ['Michelle', 8627],
                                            ['Mark', 8094]],
                    'top10_most_reviews': [['8k3aO-mPeyhbR5HUucA5aA', 13278],
                                            ['RtGqdDBvvBCjcu5dUqwfzA', 12390],
                                            ['hWDybu_KvYLSdEFzGrniTw', 10022],
                                            ['P5bUL3Engv-2z6kKohB6qQ', 9821],
                                            ['8RcEwGrFIgkt9WQ35E6SnQ', 7750],
                                            ['nmdkHL2JKFx55T3nq5VziA', 7089],
                                            ['Xwnf20FKuikiHcSpcEbpKQ', 6738],
                                            ['Hi10sGSZNxQH3NLyWSZ1oA', 6407],
                                            ['CxDOIDnH8gp9KXzpBHJYXw', 6314],
                                            ['HFECrzYDpgbS5EmTBtj2zQ', 5398]]}

In [4]:
# sssss
sc = SparkContext()
userRDD = sc.textFile(path + user_file)
userRDD_json = userRDD.map(json.loads)
# A. total number of users
user_cnt = userRDD_json.count()
print("total_users: ", user_cnt)

# B. average number of reviews
review_cnt_RDD = userRDD_json.map(lambda x: [x['user_id'],x['review_count']])
review_cnt_mean = review_cnt_RDD.values().mean()
print("avg_reviews: ", review_cnt_mean)

# C. number of distinct user names
user_nameRDD = userRDD_json.map(lambda x: (x['name'],1)).reduceByKey(lambda x,y:x+y)
user_name_cnt = user_nameRDD.count()
print("distinct_usernames: ", user_name_cnt)

# D. number of users joined in 2011
user_2011_cnt = userRDD_json.filter(lambda x: x['yelping_since'][:4] == '2011').count()
print("num_users: ", user_2011_cnt)

# E. Top 10 popular names and the number of times they 
#    appear (user names that appear the most number of times) 
most_name = user_nameRDD.sortBy(lambda x: x[1],ascending=False).take(10)
print("\n\ntop10_popular_names:")
for item in most_name:
    print(item)

# F. Find Top 10 user ids who have written the most number of reviews (0.5 point)
most_review = review_cnt_RDD.sortBy(lambda x: x[1],ascending=False).take(10)
print("\n\ntop10_most_reviews:")
for item in most_review:
    print(item)

total_users:  1637138
avg_reviews:  22.287697799452484
distinct_usernames:  124917
num_users:  168467


top10_popular_names:
('John', 14430)
('David', 13616)
('Michael', 13559)
('Chris', 12178)
('Mike', 11358)
('Jennifer', 10360)
('Jessica', 9349)
('Sarah', 8748)
('Michelle', 8627)
('Mark', 8094)


top10_most_reviews:
['8k3aO-mPeyhbR5HUucA5aA', 13278]
['RtGqdDBvvBCjcu5dUqwfzA', 12390]
['hWDybu_KvYLSdEFzGrniTw', 10022]
['P5bUL3Engv-2z6kKohB6qQ', 9821]
['8RcEwGrFIgkt9WQ35E6SnQ', 7750]
['nmdkHL2JKFx55T3nq5VziA', 7089]
['Xwnf20FKuikiHcSpcEbpKQ', 6738]
['Hi10sGSZNxQH3NLyWSZ1oA', 6407]
['CxDOIDnH8gp9KXzpBHJYXw', 6314]
['HFECrzYDpgbS5EmTBtj2zQ', 5398]


In [7]:
def task1():
    sc = SparkContext.getOrCreate()
    sc.stop()
    sc = SparkContext()
    userRDD = sc.textFile(path + user_file)
    userRDD_json = userRDD.map(json.loads)
    # A. total number of users
    user_cnt = userRDD_json.count()

    # B. average number of reviews
    review_cnt_RDD = userRDD_json.map(lambda x: [x['user_id'],x['review_count']])
    review_cnt_mean = review_cnt_RDD.values().mean()

    # C. number of distinct user names
    user_nameRDD = userRDD_json.map(lambda x: x['name']).groupBy(lambda x: x)
    user_name_cnt = user_nameRDD.count()

    # D. number of users joined in 2011
    user_2011_cnt = userRDD_json.filter(lambda x: x['yelping_since'][:4] == '2011').count()

    # E. Top 10 popular names and the number of times they 
    #    appear (user names that appear the most number of times) 
    user_name_cntRDD = user_nameRDD.map(lambda x: [x[0],len(x[1])])
    most_name = user_name_cntRDD.sortBy(lambda x: x[1],ascending=False).take(10)


    # F. Find Top 10 user ids who have written the most number of reviews (0.5 point)
    most_review = review_cnt_RDD.sortBy(lambda x: x[1],ascending=False).take(10)
    
    result_json = {'total_users': user_cnt,
                  'avg_reviews': review_cnt_mean,
                  'distinct_usernames': user_name_cnt,
                  'num_users': user_2011_cnt,
                  'top10_popular_names': most_name,
                  'top10_most_reviews': most_review}

In [11]:
def task1_2():
    sc = SparkContext.getOrCreate()
    sc.stop()
    sc = SparkContext()
    userRDD = sc.textFile(path + user_file)
    userRDD_json = userRDD.map(json.loads)
    # A. total number of users
    user_cnt = userRDD_json.count()
    

    # B. average number of reviews
    review_cnt_RDD = userRDD_json.map(lambda x: [x['user_id'],x['review_count']])
    review_cnt_mean = review_cnt_RDD.values().mean()
   
    # C. number of distinct user names
    user_nameRDD = userRDD_json.map(lambda x: (x['name'],1)).reduceByKey(lambda x,y:x+y)
    user_name_cnt = user_nameRDD.count()
    

    # D. number of users joined in 2011
    user_2011_cnt = userRDD_json.filter(lambda x: x['yelping_since'][:4] == '2011').count()
    

    # E. Top 10 popular names and the number of times they 
    #    appear (user names that appear the most number of times) 
    most_name = user_nameRDD.sortBy(lambda x: x[1],ascending=False).take(10)
    
    # F. Find Top 10 user ids who have written the most number of reviews (0.5 point)
    most_review = review_cnt_RDD.sortBy(lambda x: x[1],ascending=False).take(10)
    
    result_json = {'total_users': user_cnt,
                  'avg_reviews': review_cnt_mean,
                  'distinct_usernames': user_name_cnt,
                  'num_users': user_2011_cnt,
                  'top10_popular_names': most_name,
                  'top10_most_reviews': most_review}

In [8]:
%time task1()

CPU times: user 120 ms, sys: 30.4 ms, total: 150 ms
Wall time: 41.5 s


In [ ]:
%time task1_2()